#  SIMD programming (II) --- explicit/low-level SIMD programming with vector extension + intrinsics

# 1. Introduction
* GCC, LLVM, and NVIDIA HPC Compilers allow you to define a type representing a vector of values
* see [GCC manual](https://gcc.gnu.org/onlinedocs/gcc-9.5.0/gcc/Vector-Extensions.html#Vector-Extensions) for reference
* with this feature, you can write a code that explicitly manipulates a vector of values
* for example, the code below defines a new type, `floatv`, as a 64-byte vector of `float`s (i.e., sixteen elements of `float`s)
```
typedef float floatv __attribute__((vector_size(64)));
```
* good news is that this type supports a familiar syntax for many of arithmetic expressions (+, -, *, /, etc.)
* for other operations you can use intrinsic functions
* in this notebook you are going to experience these features

# 2. Compilers
## 2-1. Set up NVIDIA HPC SDK
Execute this before you use NVIDIA HPC SDK

In [ ]:
export PATH=/opt/nvidia/hpc_sdk/Linux_x86_64/24.9/compilers/bin:$PATH

Check if it works (check if full paths of nvc/nvc++ are shown)

In [ ]:
which nvc
which nvc++

## 2-2. Set up LLVM
Execute this before you use LLVM

In [ ]:
export PATH=/home/share/llvm/bin:$PATH

Check if it works (check if full paths of gcc/g++ are shown)

In [ ]:
which clang
which clang++

## 2-3. GCC

Check if it works (check if full paths of nvc/nvc++ are shown)

In [ ]:
which gcc
which g++

# 3. Vector extension
* the following code defines types representing a vector of four, eight, and sixteen floats (`float_4`, `float_8`, and `float_16`, respectively) and functions that takes two such values

In [ ]:
%%writefile vector_ext.c
typedef float float_4 __attribute__((vector_size(16),__may_alias__,aligned(sizeof(float))));
typedef float float_8 __attribute__((vector_size(32),__may_alias__,aligned(sizeof(float))));
typedef float float_16 __attribute__((vector_size(64),__may_alias__,aligned(sizeof(float))));

float_4 dist_4(float_4 x, float_4 y) {
  return x * x + y * y;
}
float_8 dist_8(float_8 x, float_8 y) {
  return x * x + y * y;
}
float_16 dist_16(float_16 x, float_16 y) {
  return x * x + y * y;
}


In [ ]:
BEGIN SOLUTION
END SOLUTION
# will produce vector_ext.s
clang -S -Wall -O3 -mavx512f -mfma vector_ext.c
#nvc -S -Wall -O3 -mavx512f -mfma vector_ext.c
#gcc -S -Wall -O3 -mavx512f -mfma vector_ext.c

In [ ]:
BEGIN SOLUTION
END SOLUTION
cat -n vector_ext.s

* recent Clang/NVIDIA/GCC allow arithmetic expressions mixing vectors and scalars

In [ ]:
%%writefile scalar_vector.c
typedef float float_16 __attribute__((vector_size(64),__may_alias__,aligned(sizeof(float))));

float_16 axpb_16(float a, float_16 x) {
  return a * x + 3.0;
}


In [ ]:
BEGIN SOLUTION
END SOLUTION
# will produce scalar_vector.s
clang -S -Wall -O3 -mavx512f -mfma scalar_vector.c
#nvc -S -Wall -O3 -mavx512f -mfma scalar_vector.c
#gcc -S -Wall -O3 -mavx512f -mfma scalar_vector.c

In [ ]:
BEGIN SOLUTION
END SOLUTION
cat -n scalar_vector.s

* you almost certainly want to match the size of the vector with instructions used (i.e., 64 for AVX512F and 32 for AVX or AVX2)
* for which you can use following compile-time macros to make your code somewhat portable
  * `__AVX512F__` defined when AVX512F instructions will be used
  * `__AVX2__` defined when AVX2 instructions will be used
  * `__AVX__` defined when AVX instructions will be used
  * `__SSE__` defined when SSE instructions will be used
  * etc.
* you will typically want to use the largest available vector size and if it is all you need, the following code defines `floatv` vector type and try to maintain ISA-dependent code small

In [ ]:
%%writefile def_vector_type.c
#if defined(__AVX512F__)
#warning "__AVX512F__ defined. SIMD width = 64 bytes"
enum { simd_width = 64 };
#elif defined(__AVX2__) || defined(__AVX__)
#warning "__AVX__ defined. SIMD width = 32 bytes"
enum { simd_width = 32 };
#elif defined(__SSE2__) || defined(__SSE__)
#warning "__SSE__ defined. SIMD width = 16 bytes"
enum { simd_width = 16 };
#else
#error "sorry, you must have one of __SSE__, __SSE2__, __AVX__, __AVX2__, or __AVX512F__"
#endif

typedef float floatv __attribute__((vector_size(simd_width),__may_alias__,aligned(sizeof(float))));
const int n_float_lanes = sizeof(floatv) / sizeof(float);

floatv distv(floatv x, floatv y) {
  return x * x + y * y;
}


In [ ]:
BEGIN SOLUTION
END SOLUTION
# without -mavx512f, it will use only up to AVX2 (something that will change in future and may be compiler-dependent)
clang -S -Wall -O3 -mfma def_vector_type.c
#nvc -S -Wall -O3 -mfma def_vector_type.c
#gcc -S -Wall -O3 -mfma def_vector_type.c

In [ ]:
BEGIN SOLUTION
END SOLUTION
cat -n def_vector_type.s

In [ ]:
BEGIN SOLUTION
END SOLUTION
# for today, you should give -mavx512f to make sure the compiler uses AVX512 instructions (512 bit SIMD)
clang -S -Wall -O3 -mavx512f -mfma def_vector_type.c
#nvc -S -Wall -O3 -mavx512f -mfma def_vector_type.c
#gcc -S -Wall -O3 -mavx512f -mfma def_vector_type.c

In [ ]:
BEGIN SOLUTION
END SOLUTION
cat -n def_vector_type.s

## 3-1. a note about `aligned(sizeof(float))`
* if you are curious, `aligned(sizeof(float))` specifies its minimum alignment to be the size of float (4), which is convenient as `floatv` will be used to work on an array of floats
* without it, the alignment becomes the size of `floatv` (32)
* the true effect is that the compiler uses vector load/store instructions `vmovups`, which does not assume 32 byte alignment; without it, the compiler uses `vmovaps`, vector load/store instructions that will raise a segfault when the given address is not 32 byte-aligned
* recent Intel CPUs do not impose any performance penalty on `vmovups` compared to `vmovaps`, so there is not much point in making `floatv` aligned to 32 byte

# 4. Programming in vector extension
* once you have defined a vector type, there are not much you should learn other than how to build a vector from scalars and vice versa

## 4-1.  Building a vector value from scalars
### 4-1-1. from an array of scalars
* you access an array of scalars (i.e., `float *`) with a pointer to a vector (e.g., `floatv`) and you get a vector of consecutive elements starting from the given address

In [ ]:
%%writefile loadv.c
#if defined(__AVX512F__)
enum { simd_width = 64 };
#else
#error "you must have __AVX512F__ (forgot to give -mavx512f -mfma??)"
#endif

typedef float floatv __attribute__((vector_size(simd_width),__may_alias__,aligned(sizeof(float))));
const int n_float_lanes = sizeof(floatv) / sizeof(float);

floatv loadv(float * a) {
  /* a vector {a[0],a[1],...,a[L-1]} */
  return *((floatv *)a);
}

In [ ]:
BEGIN SOLUTION
END SOLUTION
# will produce loadv.s
clang -S -Wall -O3 -mavx512f -mfma loadv.c
#nvc -S -Wall -O3 -mavx512f -mfma loadv.c
#gcc -S -Wall -O3 -mavx512f -mfma loadv.c

In [ ]:
BEGIN SOLUTION
END SOLUTION
cat -n loadv.s

### 4-1-2. building a vector of arbitrary values
* you can build a vector of arbitrary values just by making an array of these values and by applying the method above

In [ ]:
%%writefile make_vector.c
#if defined(__AVX512F__)
enum { simd_width = 64 };
#else
#error "you must have __AVX512F__ (forgot to give -mavx512f -mfma??)"
#endif

typedef float floatv __attribute__((vector_size(simd_width),__may_alias__,aligned(sizeof(float))));
const int n_float_lanes = sizeof(floatv) / sizeof(float);

floatv make_vector(float a0, float a1, float a2, float a3,
                   float a4, float a5, float a6, float a7,
                   float a8, float a9, float a10, float a11,
                   float a12, float a13, float a14, float a15
                   ) {
  float a[16] = { a0, a1, a2, a3, a4, a5, a6, a7,
    a8, a9, a10, a11, a12, a13, a14, a15
  };
  return *((floatv *)a);
}

In [ ]:
BEGIN SOLUTION
END SOLUTION
# will produce make_vector.s
clang -S -Wall -O3 -mavx512f -mfma make_vector.c
#nvc -S -Wall -O3 -mavx512f -mfma make_vector.c
#gcc -S -Wall -O3 -mavx512f -mfma make_vector.c

In [ ]:
BEGIN SOLUTION
END SOLUTION
cat -n make_vector.s

* you will later learn there is an intrinsic function exactly for it

## 4-2. Extracting a scalar value from a vector
### 4-2-1. to an array of scalars
* you can store a vector into an array of scalars

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile storev.c
#if defined(__AVX512F__)
enum { simd_width = 64 };
#else
#error "you must have __AVX512F__ (forgot to give -mavx512f -mfma??)"
#endif

typedef float floatv __attribute__((vector_size(simd_width),__may_alias__,aligned(sizeof(float))));
const int n_float_lanes = sizeof(floatv) / sizeof(float);

void storev(float * a, floatv v) {
  *((floatv *)a) = v;
}

In [ ]:
BEGIN SOLUTION
END SOLUTION
# will produce storev.s
clang -S -Wall -O3 -mavx512f -mfma storev.c
#nvc -S -Wall -O3 -mavx512f -mfma storev.c
#gcc -S -Wall -O3 -mavx512f -mfma storev.c

In [ ]:
BEGIN SOLUTION
END SOLUTION
cat -n storev.s

### 4-2-2. index notation
* a vector's element can be extracted using an index notation for arrays, but internally, it typically stores the vector into a temporary array and then extracts the element you want, which is not terribly efficient

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile get_i.c
#if defined(__AVX512F__)
enum { simd_width = 64 };
#else
#error "you must have __AVX512F__ (forgot to give -mavx512f -mfma??)"
#endif

typedef float floatv __attribute__((vector_size(simd_width),__may_alias__,aligned(sizeof(float))));
const int n_float_lanes = sizeof(floatv) / sizeof(float);

float get_i(floatv v, int i) {
  return v[i];
}

In [ ]:
BEGIN SOLUTION
END SOLUTION
# will produce get_i.s
clang -S -Wall -O3 -mavx512f -mfma get_i.c
#nvc -S -Wall -O3 -mavx512f -mfma get_i.c
#gcc -S -Wall -O3 -mavx512f -mfma get_i.c

In [ ]:
BEGIN SOLUTION
END SOLUTION
cat -n get_i.s

# 5. Vector intrinsics
## 5-1. When you have to use vector intrinsics
* vector intrinsics are functions almost directly compiled to corresponding SIMD instructions
* remember that with vector extension, you can have a vector of values and write arithmetic expressions involving them; you do not have to use vector intrinsics where it is applicable
* things you wish you could do with it but cannot include
  * comparisons between two vectors; given `A` and `B` of type `floatv`, one wishes `A < B` a vector of booleans or ints {A[0]<B[0], A[1]<B[1], ..., A[L-1]<B[L-1]} but it doesn't
  * math functions such as sqrt, log, exp, etc.; given `V` of type `floatv`, one wishes `sqrt(V)` returns a value of `floatv` applying sqrt on each element of `V` but it doesn't (a SIMD sqrt instruction in fact exists on recent Intel CPUs)
  * array indexing using a vector of integers; given `I` of type `intv` (a vector of ints, whose definition will follow the definition of `floatv` above) and `a` of `float*`, one wishes `a[I]` to be a value of `floatv` {a[I[0]], a[I[1]], ..., a[I[L-1]]} (what a gather instruction does on Intel CPUs supporting AVX2)
* in all these circumstances, you need vector intrinsics

## 5-2. Vector intrinsics introduction
* https://software.intel.com/sites/landingpage/IntrinsicsGuide/
* you will be overwhelmed by the sheer number of functions; never have to remember all functions that exist
* just remember that
  * most functions correspond to a SIMD instruction, so what you reasonably hope will exist hopefully exists
    * e.g., wish to do SIMD sqrt? just search for sqrt and look for what you need
  * Intel keeps extending instruction set and SIMD width, so there are multiple versions doing a similar thing

* with that said, here are an overview

### 5-2-1. The header file
* You should include 
```
#include <x86intrin.h>
```

### 5-2-2. Types
* type names begin with __m followed by the number of bits (128, 256 or 512), possibly followed by a character indicating the element type

* `__m128`, `__m256`, `__m512` : vector of floats, with size 128, 256, and 512 bits, respectively.  they are presumably defined exactly like floatv we have introduced before
* `__m128d`, `__m256d`, `__m512d` : vector of doubles, with size 128, 256, and 512 bits, respectively
* `__m128i`, `__m256i`, `__m512i` : vector of ints, with size 128, 256, and 512 bits, respectively

### 5-2-3. Function names
* function names begin with _mm followed by the number of bits (128 bit versions lack this part). e.g.,
  * `__m512 _mm512_fmadd_ps (__m512 a, __m512 b, __m512 c)` takes three 512 bit vectors and perform fmadd (a * b + c)

### 5-2-4. set and set1
* `_mm_set_ps`, `_mm256_set_ps` and `_mm512_set_ps` take 4, 8 and 16 float values and returns a vector of them, respectively
* `_mm_set1_ps`, `_mm256_set1_ps` and `_mm512_set1_ps` takes a float value and returns a 4, 8 and 16-element vector of it. e.g., `_mm_set1_ps(3)` returns `{3,3,3,3}`
* similar functions exist for doubles (e.g., `_mm512_set_pd` takes eight double values)

* they are special in that there are actually no instructions directly corresponding them; they are provided for convenience and implemented by a sequence of operations. you should not assume they are as efficient as a single instruction

### 5-2-5. A brief categorization
Inside parentheses is the category you want to check at the Intel Intrinsics Guide page

* arithmetic, which you can do just with vector extension without using intrinsics (Arithmetic)
* math functions (Elementary Math)
* comparison (Compare)
* shuffling values on vectors (Bit Manipulation)
* memory access (Load and Store)

# 6. Manual vectorization with vector types and intrinsics
## 6-1. Loops containing branches
* the basic template for a loop containing branch (if expression), like this
```
for (i = 0; i < n; i++) {
    if (C(i)) {
      T(i);
    } else {
      E(i);
    }
}  
```
is
```
for (i = 0; i < n; i += L) {
    k = C(i:i+L); // L bit mask
    T(i:i+L) predicated on k;
    E(i:i+L) predicated on ~k;
}  
```

* to realize this, you need
  * compare instructions to calculate the mask
  * predicated version of many instructions
  * instructions that blend values from two SIMD values

* search [Intrinsics guide](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html) for "cmp" to know the flavor of compare instructions, 
* "mm512_mask_" to know the flavor of various predicated instructions (e.g., mm512_mask_add), and
* "mm512_mask_blend" to know the flavor of value-blending instructions

# <font color="green"> Problem 1 :  Vectorize integral</font>
* the following code calculates
$$ \int_0^1 \int_0^1 f(x,y)\,dx\,dy $$
where
$$ f(x,y) = \left\{\begin{array}{ll} \sqrt{1 - x^2 - y^2} & (1 - x^2 - y^2 > 0) \\ 0 & \mbox{otherwise} \end{array}\right. $$

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile integral_no_simd.c
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

/* scalar version */
double int_sqrt_one_minus_x2_y2(long n) {
  double h = 1.0 / n;
  double s = 0.0;
  for (long i = 0; i < n; i++) {
    for (long j = 0; j < n; j++) {
      double x = i * h ;
      double y = j * h;
      double z = 1 - x * x - y * y;
      if (z > 0.0) {
        s += sqrt(z);
      } else {
        break;
      }
    }
  }
  return s * h * h;
}

// wall clock time
double get_wtime() {
  struct timespec ts[1];
  clock_gettime(CLOCK_REALTIME, ts);
  return ts->tv_sec + ts->tv_nsec * 1.0e-9;
}

int main(int argc, char ** argv) {
  int i = 1;
  long n      = (argc > i ? atof(argv[i]) : 30L * 1000L); i++;
  long repeat = (argc > i ? atof(argv[i]) : 5);           i++;
  n += 15;
  n = (n / 16) * 16;
  double n_sqrts = (M_PI / 4) * n * n;
  for (long r = 0; r < repeat; r++) {
    printf("===== repeat %ld =====\n", r);
    fflush(stdout);
    double t0 = get_wtime();
    double s = int_sqrt_one_minus_x2_y2(n);
    double t1 = get_wtime();
    double dt = t1 - t0;
    printf("s = %.9f (err = |s - pi/6| = %e)\n", s, fabs(s - M_PI/6));
    printf("%f sec\n", dt);
    printf("%f points/sec\n", n_sqrts / dt);
    fflush(stdout);
  }
  return 0;
}

In [ ]:
BEGIN SOLUTION
END SOLUTION
clang -Wall -O3 -mavx512f -mfma -o integral_no_simd integral_no_simd.c -lm
#nvc -Wall -O3 -mavx512f -mfma -o integral_no_simd integral_no_simd.c -lm
#gcc -Wall -O3 -mavx512f -mfma -o integral_no_simd integral_no_simd.c -lm

In [ ]:
BEGIN SOLUTION
END SOLUTION
./integral_no_simd 10000
./integral_no_simd 30000

* vectorize it by changing the following code
* the key is how to vectorize despite the presence of the branch 

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile integral_simd.c
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

double int_sqrt_one_minus_x2_y2(long n) {
  double h = 1.0 / n;
  double s = 0.0;
  for (long i = 0; i < n; i++) {
    for (long j = 0; j < n; j++) {
      double x = i * h ;
      double y = j * h;
      double z = 1 - x * x - y * y;
      if (z > 0.0) {
        s += sqrt(z);
      } else {
        break;
      }
    }
  }
  return s * h * h;
}

// wall clock time
double get_wtime() {
  struct timespec ts[1];
  clock_gettime(CLOCK_REALTIME, ts);
  return ts->tv_sec + ts->tv_nsec * 1.0e-9;
}

int main(int argc, char ** argv) {
  int i = 1;
  long n      = (argc > i ? atof(argv[i]) : 30L * 1000L); i++;
  long repeat = (argc > i ? atof(argv[i]) : 5);           i++;
  n += 15;
  n = (n / 16) * 16;
  double n_sqrts = (M_PI / 4) * n * n;
  for (long r = 0; r < repeat; r++) {
    printf("===== repeat %ld =====\n", r);
    fflush(stdout);
    double t0 = get_wtime();
    double s = int_sqrt_one_minus_x2_y2(n);
    double t1 = get_wtime();
    double dt = t1 - t0;
    printf("s = %.9f (err = |s - pi/6| = %e)\n", s, fabs(s - M_PI/6));
    printf("%f sec\n", dt);
    printf("%f points/sec\n", n_sqrts / dt);
    fflush(stdout);
  }
  return 0;
}

In [ ]:
BEGIN SOLUTION
END SOLUTION
clang -Wall -O3 -mavx512f -mfma -o integral_simd integral_simd.c -lm
#nvc -Wall -O3 -mavx512f -mfma -o integral_simd integral_simd.c -lm
#gcc -Wall -O3 -mavx512f -mfma -o integral_simd integral_simd.c -lm

In [ ]:
BEGIN SOLUTION
END SOLUTION
prlimit --cpu=3.0 ./integral_simd 10000
prlimit --cpu=10.0 ./integral_simd 30000

* <font color="red">note:</font> `prlimit --cpu=x` limits the CPU time the command is allowed to spend to `x` sec
* this is for our safety net when your program goes terribly wrong
* you are encouraged to use it always when you are not sure, even when it is not given by default (I easily forget to put it)
* you can change the time as necessary, but default values give you a clue about the expectation (if your program is killed, chances are your program is slower than it should be)

## 6-2. Loops containing non-contiguous memory access
* when you vectorize a scalar loop
```
for (i = 0; i < n; i++) {
    S(i);
}  
```
into something like
```
for (i = 0; i < n; i += L) {
    S(i:i+L);
}  
```
an issue arises when consecutive iterations, say S(i) and S(i+1), access non-contiguous elements by the same instruction, like this.
```
for (i = 0; i < n; i++) {
    ... a[2 * i] ...
}  
```

* to implement an expression `a[2 * i]`, we wish to have an instruction that accesses `a[2*i], a[2*i+2], a[2*i+4], ...`
* in a vector extension, you can easily access consecutive elements `a[2*i], a[2*i+1], a[2*i+2], ...` for an array of integers (int or long) or floating point numbers (float or double), just by dereferencing a pointer to an appropriate vector type, but there is no convenient expression to access non-contiguous elements

* Intel recently introduced gather/scatter instructions that just do that (gather for load and scatter for store)
* search [Intrinsics guide](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html) for "gather" or "scatter" to know their flavors

# <font color="green"> Problem 2 :  Vectorize binary search</font>
* <font color="red">WARNING: challenges ahead</font>
* binary search is a method to efficiently search a sorted array
* it is a very sequential algorithm, so there is no way to vectorize (or parallelize, for that matter) searching for a single element
* but when given many elements to search for, we naturally wish to vectorize it, which is what you will be doing 
* the function that takes a single value to search for (`binsearch`) and the outer loop that searches for many elements (`binsearch_many`) look like this

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile binsearch_nosimd.h
/* search a[0:n] looking for value x,
   assuming a is sorted.
   return an integer p s.t.
   a[p] <= x < a[p+1]
   (a[-1] is considered smaller than any value and 
    a[n] larger than any value)
 */
int binsearch(int * a, int n, int x) {
  if (x < a[0]) return -1;
  if (a[n-1] <= x) return n - 1;
  /* a[0] <= x < a[n-1]  */
  int p = 0, q = n - 1;
  while (q - p > 1) {
    /* INV: a[p] <= x < a[q] */
    int r = (p + q) / 2;
    if (a[r] <= x) p = r;
    else q = r;
  }
  /* a[p] <= x < a[p+1] */
  return p;
}

/* search a[0:n] for all values in x[0:m]
   and return the number of elements found */
int binsearch_many(int * a, int n, int * x, int m) {
  int c = 0;
  for (int i = 0; i < m; i++) {
    int p = binsearch(a, n, x[i]);
    assert(-1 <= p);
    assert(p == -1 || a[p] <= x[i]);
    assert(p < n);
    assert(p == n - 1 || x[i] < a[p + 1]);
    if (0 <= p && a[p] == x[i]) c++; // found? -> c++
  }
  return c;
}


* your goal is to vectorize `binsearch(int * a, int n, int x)`, so that it now takes a vector of (16) integers and returns another vector, and `int binsearch_many(int * a, int n, int * x, int m)`, so that it now passes a vector of (16) integers at a time rather than one
* it is very challenging, as `binsearch` itself contains a loop, branches, non-contiguous accesses, everything that challenges vectorizing compilers (and human)
* here is the skeleton code that emits an error when executed (modify this cell)

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile binsearch_simd.h
intv binsearch_simd(int * a, int n, intv x) {
  intv p = intv(0);
  intv q = intv(n - 1);
  mask k0 = x < a[0];
  mask k1 = a[n-1] <= x;
  p = blend(k0,    intv(-1), p);
  p = blend(k1, intv(n - 1), p);
  mask k = ~k0 & ~k1;
  k &= (q - p > 1);
  while (any(k)) {
    /* INV: a[p] <= x < a[q] */
    intv r = (p + q) / 2;
    mask c = gather(a, r, k) <= x;
    p = blend( c & k, r, p);
    q = blend(~c & k, r, q);
    k &= (q - p > 1);
  }
  /* a[p] <= x < a[p+1] */
  return p;
}

int binsearch_many_simd(int * a, int n, int * x, int m) {
  int c = 0;
  assert(L == 16);
  for (int i = 0; i < m; i += L) {
    intv xv = intv(-1);
    intv li = linear(i);
    xv = blend(li < m, V(x[i]), xv);
    intv p = binsearch_simd(a, n, xv);
    assert(all(-1 <= p));
    assert(all((p == -1)    | (gather(a, p, 0 <= p) <= xv)));
    assert(all(p < n));
    assert(all((p == n - 1) | (xv < gather(a, p + 1, p < n - 1))));
    c += count_one((0 <= p) & (gather(a, p, 0 <= p) == xv));
  }
  return c;
}


* the problem is challenging as you have to use so many intrinsics and their names are so long and obscure that they will damage your brain (e.g., `(a < b)` becomes `_mm512_cmp_epi32_mask(a, b, _MM_CMPINT_LT)`)
* to help circumvent this issue, C++ operator overloading can help you a lot, to make the code much more "ordinary"
* I gave a header file, `binsearch_simd_util.h` specifically tailored for this problem
* you might want to add your own utility function there too, but I do not expect that is necessary

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile binsearch_simd_util.h
/* a convenient type definition and overloaded operators */

/* an aux type representing 16 ints, defined in the way suggested in slides */
typedef int _intv __attribute__((vector_size(64), __may_alias__, aligned(sizeof(int))));
enum { L = sizeof(_intv) / sizeof(int) };

/* a data structure that looks vector operations look nice 
   using C++ operator overloading */

/* this is the type you are going to use to represent 16 ints */
struct intv {
  /* takes 16 values */
  intv(_intv _v) { v = _v; }
  /* takes a single value (e.g., intv(3)) */
  intv(int _v) { v = _mm512_set1_epi32(_v); }
  _intv v;
};
/* 16 bit mask representing the result of a comparison */
typedef __mmask16 mask;

// C++ trick so that (a == b) returns the mask
mask operator==(intv a, intv b) {
  mask k = _mm512_cmp_epi32_mask(a.v, b.v, _MM_CMPINT_EQ);
  return k;
}

// ditto, but accept a scalar 
mask operator==(intv a, int b) {
  mask k = a == intv(b);
  return k;
}

// ditto, but accept a scalar 
mask operator==(int a, intv b) {
  mask k = intv(a) == b;
  return k;
}

// C++ trick so that (a < b) returns the mask
mask operator<(intv a, intv b) {
  mask k = _mm512_cmp_epi32_mask(a.v, b.v, _MM_CMPINT_LT);
  return k;
}

mask operator<(intv a, int b) {
  mask k = a < intv(b);
  return k;
}

mask operator<(int a, intv b) {
  mask k = intv(a) < b;
  return k;
}

// C++ trick so that (a > b) returns the mask
mask operator>(intv a, intv b) {
  mask k = _mm512_cmp_epi32_mask(a.v, b.v, _MM_CMPINT_NLE);
  return k;
}

mask operator>(intv a, int b) {
  mask k = a > intv(b);
  return k;
}

mask operator>(int a, intv b) {
  mask k = intv(a) > b;
  return k;
}

// C++ trick so that (a <= b) returns the mask
mask operator<=(intv a, intv b) {
  mask k = _mm512_cmp_epi32_mask(a.v, b.v, _MM_CMPINT_LE);
  return k;
}

mask operator<=(intv a, int b) {
  mask k = a <= intv(b);
  return k;
}

mask operator<=(int a, intv b) {
  mask k = intv(a) <= b;
  return k;
}

// blend a and b by mask value
// k[i] is 1 -> take a[i]
// k[i] is 0 -> take b[i]
// read it as if it is C expression (k ? a : b)
intv blend(mask k, intv a, intv b) {
  _intv v = _mm512_mask_blend_epi32(k, b.v, a.v);
  return intv(v);
}

// a + b (add other versions if necessary)
intv operator+(intv a, intv b) {
  _intv v = a.v + b.v;
  return intv(v);
}

// a - b (add other versions if necessary)
intv operator-(intv a, intv b) {
  intv v = a.v - b.v;
  return intv(v);
}

// a * b (add other versions if necessary)
intv operator*(intv a, intv b) {
  intv v = a.v * b.v;
  return intv(v);
}

// a / b (add other versions if necessary)
intv operator/(intv a, intv b) {
  _intv v = a.v / b.v;
  return intv(v);
}

intv operator/(intv a, int b) {
  intv v = a / intv(b);
  return v;
}

// gather(a, I, k) returns
// a[I0], a[I1], ..., a[IL-1] for lanes whose bit is set in k.
// unset lanes become zero
intv gather(int * a, intv i, mask k) {
  _intv y = _mm512_mask_i32gather_epi32(intv(0).v, k, i.v, a, sizeof(int));
  return intv(y);
}

// return true if all bits of k are set
int all(mask k) {
  return (0xffff & k) == 0xffff;
}

// return true if any bit of k is set
int any(mask k) {
  return k != 0;
}

// count the number of bits set in k
int count_one(mask k) {
  int x = _mm_popcnt_u32((unsigned int)k);
  return x;
}

// access consecutive L locations starting from an int location 
intv& V(int& p) {
  return *((intv*)&p);
}

// [a, a+1, a+2, ..., a+L-1]
intv linear(int a) {
  int v[L];
  for (int i = 0; i < L; i++) {
    v[i] = a + i;
  }
  return intv(*((_intv*)v));
}


* the main function that generates an array and measure time

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile binsearch_main.cc
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <x86intrin.h>

#include "binsearch_nosimd.h"
#include "binsearch_simd_util.h"
#include "binsearch_simd.h"

// make a random array of n elements, each element in [0:max_val)
int * make_random_array(int n, int max_val, unsigned long seed) {
  int * a = (int *)malloc(sizeof(int) * n);
  assert(a);
  unsigned short rg[3] = {
    (unsigned short)(seed >> 32),
    (unsigned short)(seed >> 16),
    (unsigned short)(seed >>  0)
  };
  for (int i = 0; i < n; i++) {
    a[i] = nrand48(rg) % max_val;
  }
  return a;
}

// print a[0:n] (don't call it with large n)
void print_array(int * a, int n) {
  for (int i = 0; i < n; i++) {
    printf("%d ", a[i]);
  }
  printf("\n");
}

// wall clock time
double get_wtime() {
  struct timespec ts[1];
  clock_gettime(CLOCK_REALTIME, ts);
  return ts->tv_sec + ts->tv_nsec * 1.0e-9;
}

// aux function to sort the searched array
int cmp_int(const void * _a, const void * _b) {
  const int * a = (const int *)_a;
  const int * b = (const int *)_b;
  return *a - *b;
}

int main(int argc, char ** argv) {
  int i = 1;
  int simd = (argc > i ? atoi(argv[i]) : 0); i++;
  int n = (argc > i ? atoi(argv[i]) : 10 * 1000 * 1000); i++;
  int m = (argc > i ? atoi(argv[i]) :  5 * 1000 * 1000); i++;
  // max_val = 3n => approximately 1/3 of values will be found
  int max_val = (argc > i ? atoi(argv[i]) : 3 * n); i++;
  unsigned long seed0 = (argc > i ? atol(argv[i]) : 1234567890L); i++;
  unsigned long seed1 = (argc > i ? atol(argv[i]) : 2345678909L); i++;

  // generate array A (to be searched)
  printf("generating array A to be searched (%d elements in range 0..%d) ...\n", n, max_val); fflush(stdout);
  int * a = make_random_array(n, max_val, seed0);
  printf("sorting ...\n"); fflush(stdout);
  qsort(a, n, sizeof(int), cmp_int);
  //print_array(a, n);

  // generate array X (values to search for)
  printf("generating values X to search for (%d elements in range 0..%d) ...\n", m, max_val); fflush(stdout);
  int * x = make_random_array(m, max_val, seed1);
  //print_array(x, m);

  // start the real thing
  printf("start binary search ...\n"); fflush(stdout);
  double t0 = get_wtime();
  int c = (simd == 0 ?
           binsearch_many(a, n, x, m) :
           binsearch_many_simd(a, n, x, m));
  double t1 = get_wtime();
  printf("done in %f sec\n", t1 - t0);
  printf("%d out of %d elements found in the array\n", c, m);
  free(a);
  free(x);
  return 0;
}

In [ ]:
BEGIN SOLUTION
END SOLUTION
clang++ -Wall -O3 -mavx512f -mfma -o binsearch_main binsearch_main.cc
#nvc++ -Wall -O3 -mavx512f -mfma -o binsearch_main binsearch_main.cc
#g++ -Wall -O3 -mavx512f -mfma -o binsearch_main binsearch_main.cc

In [ ]:
BEGIN SOLUTION
END SOLUTION
prlimit --cpu=10.0 ./binsearch_main

* if you give 1 to the first command line argument, it executes SIMD version, which is of course not implemented

In [ ]:
BEGIN SOLUTION
END SOLUTION
prlimit --cpu=10.0 ./binsearch_main 1

* default parameters are the following and can be specified as second and third parameters
  * the size of the array to be searched = 10 * 1000 * 1000
  * the number of elements to search for = 5 * 1000 * 1000

In [ ]:
BEGIN SOLUTION
END SOLUTION
# search 10-element array for 3 values, without SIMD
prlimit --cpu=10.0 ./binsearch_main 0 10 3

* for the same parameters, scalar version and SIMD version must find exactly the same number of elements
* <font color="green">your job is to make that happen and make SIMD version run pleasingly faster (you cannot make it 16x times faster), by modifying `binsearch_nosimd.h` above</font>
* <font color="red">as I said in the beginning, this problem is very challenging
* if you find it too challenging, work on the next problem first, which is easier (and more rewarding :-)
* you will find it more efficient to work outside the browser using SSH + your favorite text editor + command line; feel free to do so
  * when doing so, be careful not to overwrite your file by hitting SHIFT + ENTER on the code cell; work on a renamed file and paste it into the cell after everything is done


In [ ]:
BEGIN SOLUTION
END SOLUTION
n=$((10 * 1000 * 1000))
m=$((5 * 1000 * 1000))
echo "===== no simd ====="
prlimit --cpu=10.0 ./binsearch_main 0 ${n} ${m}  # no simd
echo "===== simd ====="
prlimit --cpu=10.0 ./binsearch_main 1 ${n} ${m}  # simd (you must make it)

# 7. Combining vectorization + parallelization
* now you learned both parallelization and vectorization
* why don't you do both?

# <font color="green"> Problem 3 :  Vectorize + parallelize integral</font>
* apply both simd and parallel for to the following
* you don't have to stick with the given parameter (10000 or 30000); play with it to observe good performance

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile integral_parallel_simd.c
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

double int_sqrt_one_minus_x2_y2(long n) {
  double h = 1.0 / n;
  double s = 0.0;
  for (long i = 0; i < n; i++) {
    for (long j = 0; j < n; j++) {
      double x = i * h ;
      double y = j * h;
      double z = 1 - x * x - y * y;
      if (z > 0.0) {
        s += sqrt(z);
      } else {
        break;
      }
    }
  }
  return s * h * h;
}

// wall clock time
double get_wtime() {
  struct timespec ts[1];
  clock_gettime(CLOCK_REALTIME, ts);
  return ts->tv_sec + ts->tv_nsec * 1.0e-9;
}

int main(int argc, char ** argv) {
  int i = 1;
  long n      = (argc > i ? atof(argv[i]) : 30L * 1000L); i++;
  long repeat = (argc > i ? atof(argv[i]) : 5);           i++;
  n += 15;
  n = (n / 16) * 16;
  double n_sqrts = (M_PI / 4) * n * n;
  for (long r = 0; r < repeat; r++) {
    printf("===== repeat %ld =====\n", r);
    fflush(stdout);
    double t0 = get_wtime();
    double s = int_sqrt_one_minus_x2_y2(n);
    double t1 = get_wtime();
    double dt = t1 - t0;
    printf("s = %.9f (err = |s - pi/6| = %e)\n", s, fabs(s - M_PI/6));
    printf("%f sec\n", dt);
    printf("%f points/sec\n", n_sqrts / dt);
    fflush(stdout);
  }
  return 0;
}

In [ ]:
BEGIN SOLUTION
END SOLUTION
clang -Wall -O3 -mavx512f -mfma -o integral_parallel_simd integral_parallel_simd.c -lm
#nvc -Wall -O3 -mavx512f -mfma -o integral_parallel_simd integral_parallel_simd.c -lm
#gcc -Wall -O3 -mavx512f -mfma -o integral_parallel_simd integral_parallel_simd.c -lm

In [ ]:
BEGIN SOLUTION
END SOLUTION
prlimit --cpu=3.0 ./integral_parallel_simd 10000
prlimit --cpu=10.0 ./integral_parallel_simd 30000